# Pretrained_Model
https://www.kaggle.com/code/hannes82/pppm-deberta-v3-large-closing-the-cv-lb-gap/notebook?scriptVersionId=91284532.

In [ ]:
import os

INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
import os
import gc
import re
import ast
import datetime
from sklearn.model_selection import StratifiedKFold
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import BayesianRidge
import sys
import copy
import json
import time
import math
import shutil
import string
import pickle
import random
import joblib
import itertools
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class CFG:
    num_workers=4
    path="../input/pppm-deberta-v3-large-closing-the-cv-lb-gap/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=133
#    seed=42
    seed=41
    n_fold=4
    trn_fold=[0, 1, 2, 3]

In [ ]:
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [ ]:
oof_df = pd.read_pickle(CFG.path+'oof_df.pkl')
labels = oof_df['score'].values
preds = oof_df['pred'].values
score = get_score(labels, preds)
LOGGER.info(f'CV Score: {score:<.4f}')

In [ ]:
test = pd.read_csv(INPUT_DIR+'test.csv')
submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
train_df = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(test.head())
display(submission.head())

In [ ]:
cpc_texts = torch.load(CFG.path+"cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)
display(test.head())
train_df['context_text'] = train_df['context'].map(cpc_texts)

In [ ]:
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(test.head())
train_df['text'] = train_df['anchor'] + '[SEP]' + train_df['target'] + '[SEP]'  + train_df['context_text']

In [ ]:
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

In [ ]:
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs


In [ ]:
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return feature,output

In [ ]:
def get_embeddings(df,path,plot_losses=True, verbose=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{device} is used")
            
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(path,
                       map_location=torch.device('cuda'))
    model.load_state_dict(state['model'])
    model.to(device)
    model.eval()
    
    tokenizer = AutoTokenizer.from_pretrained('../input/pppm-deberta-v3-large-closing-the-cv-lb-gap/tokenizer/')
    
    ds = TestDataset(CFG,df)
    dl = DataLoader(ds,
                  batch_size = 64,
                  shuffle=False,
                  num_workers = 4,
                  pin_memory=True,
                  drop_last=False
                 )
   
    tk0 = tqdm(dl, total=len(dl))
    embeddings = list()
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            
            feat,outputs = model(inputs)
            feat = feat.detach().cpu().numpy()
            embeddings.extend(feat)
    return np.array(embeddings)

In [ ]:
train_embeddings1 =  np.load('../input/train-embeddings/Train_embeddings1.npy')
test_embeddings1 = get_embeddings(test,'../input/pppm-deberta-v3-large-closing-the-cv-lb-gap/microsoft-deberta-v3-large_fold0_best.pth')
train_embeddings2 =  np.load('../input/train-embeddings/Train_embeddings2.npy')
test_embeddings2 = get_embeddings(test,'../input/pppm-deberta-v3-large-closing-the-cv-lb-gap/microsoft-deberta-v3-large_fold1_best.pth')
train_embeddings3 =  np.load('../input/train-embeddings/Train_embeddings3.npy')
test_embeddings3 = get_embeddings(test,'../input/pppm-deberta-v3-large-closing-the-cv-lb-gap/microsoft-deberta-v3-large_fold2_best.pth')
train_embeddings4 =  np.load('../input/train-embeddings/Train_embeddings4.npy')
test_embeddings4 = get_embeddings(test,'../input/pppm-deberta-v3-large-closing-the-cv-lb-gap/microsoft-deberta-v3-large_fold3_best.pth')

In [ ]:
import scipy
def calculate_metrics(targets, outputs):
    
    return dict(pearson=scipy.stats.pearsonr(outputs, targets)[0])

In [ ]:
preds = []
train_scores = []

df_oof=train_df.copy()
df_oof['oof'] = 0

skf = StratifiedKFold(5, shuffle=True, random_state=42)
train_df["score_bin"] = pd.cut(train_df["score"], bins=5, labels=False)
splits = list(skf.split(X=train_embeddings1, y=train_df["score_bin"].values,groups=train_df["anchor"].values))

# predicting out of fold scores for each fold and doing predictions for each training set

for i, (train_idx, val_idx) in enumerate(splits):
    print(f'\n------------- Training Fold {i + 1} / {10}')
    print("Current Time =", datetime.now().strftime("%H:%M:%S"))

    clf = BayesianRidge(n_iter=300, verbose=True)
    clf.fit(train_embeddings1[train_idx],train_df.score_bin[train_idx])
    train_score=calculate_metrics(train_df.score_bin[train_idx], clf.predict(train_embeddings1[train_idx]))
    train_scores.append(train_score)
    print(f"Fold {i} train RMSE: {train_score}")
    
    
    preds.append(clf.predict(test_embeddings1))
    x=clf.predict(train_embeddings1[val_idx])
    df_oof['oof'].iloc[val_idx]+= x




In [ ]:
y_pred = np.mean(preds,0)

In [ ]:
def create_submission(ids, predictions, path="submission.csv"):
    submission = pd.DataFrame({
        "id": ids,
        "score": predictions,
    })
    
    submission.to_csv(path, index=False)
    return submission
def merge_cpc_codes_with_data_frame(data_frame, cpc_codes, sep=" "):
    data_frame = data_frame.merge(cpc_codes, left_on="context", right_on="code")
    data_frame["text"] = (data_frame["anchor"] + sep + data_frame["title"]).str.lower()
    return data_frame


In [ ]:
test['predicted'] = y_pred

In [ ]:
sample_sub = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/sample_submission.csv')

In [ ]:
test.head()

In [ ]:
sample_sub['score'] = test['predicted']

In [ ]:
sample_sub.to_csv('submission.csv',index=False)